In [10]:
import os
import lmdb
import pickle
import time
import pandas as pd
import numpy as np
from rdkit import Chem
from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  
import warnings
warnings.filterwarnings(action='ignore')
from multiprocessing import Pool
import torch
import io 
import datetime

In [11]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

def get_csv_results(predict_path, csv_path):
    with open(predict_path, 'rb') as f:
        predict = CPU_Unpickler(f).load()
    smi_list, predict_list = [], []
    target_list = []
    for batch in predict:
        sz = batch["bsz"]
        for i in range(sz):
            smi_list.append(batch["smi_name"][i])
            predict_list.append(batch["predict"][i][0].cpu().tolist())
            target_list.append(batch["target"][i][0].cpu().tolist())
    predict_df = pd.DataFrame({"SMILES": smi_list, "predict": predict_list, "target": target_list})
    predict_df = predict_df.groupby("SMILES").mean().reset_index()
    predict_df.to_csv(csv_path,index=False)
    return predict_df

i=0
predict_path=f"/scratch/ssd004/datasets/cellxgene/unimol_pred/pred1920/{i}/weights_db.out.pkl"  # replace to your results path
csv_path=f"/scratch/ssd004/datasets/cellxgene/unimol_pred/pred1920/{i}/pred.csv"
predict_df = get_csv_results(predict_path, csv_path)
predict_df

,SMILES,predict,target
0,C#CCCCCCCCCC(=O)N(C(C(=O)NC(C)(C)C)C(CCCCCC)CC...,-0.116910,-1.0
1,C#CCCCCCCCCC(=O)N(C(C(=O)NC(C)(C)CC(C)(C)C)C(C...,-0.080322,-1.0
2,C#CCCCCCCCCC(=O)N(C(C(=O)NC)C(CCCCCC)CCCCCCCC)...,-0.092393,-1.0
3,C#CCCCCCCCCC(=O)N(C(C(=O)NC12C[C@H]3C[C@@H](C1...,-0.043090,-1.0
4,C#CCCCCCCCCC(=O)N(C(C(=O)NC1CCCC1)C(CCCCCC)CCC...,-0.086498,-1.0
...,...,...,...
22113,CCCCCCCCCCOC(=O)CCCCC(=O)N(CCN1CCNCC1)C(CCCCCC...,-0.071425,-1.0
22114,CCCCCCCCCCOC(=O)CCCCC(=O)N(CCN1CCNCC1)C(CCCCCO...,-0.034151,-1.0
22115,CCCCCCCCCCOC(=O)CCCCC(=O)N(CCN1CCNCC1)C(CCCCCO...,-0.055894,-1.0
22116,CCCCCCCCCCOC(=O)CCCCC(=O)N(CCN1CCNCC1)C(CCCCCO...,-0.020572,-1.0


In [12]:
predict_df.to_csv(f"/h/pangkuan/dev/SDL-LNP/model/unimol/notebooks/temp-valid/{i}-{datetime.datetime.now()}.csv")